# SPECFEM Users Workshop -- Day 2 (Oct. 6, 2022)
## Day 2b: Kernel Exercise


- In this notebook we will have participants run their own adjoint simulation using Day 2a as a guide 
- We will use two homogeneous halfspace models for simplicity, building on the exercise from Day 1b  
- Adjoint simulations are key for performing seismic imaging (Day 3) as their results guide iterative model updates during the inverse problem  
- These instructions should be run from inside the Docker container, using Jupyter Lab (see *Docker Preamble* in Day 0). 

-----------

**Relevant Links:** 
- Today's Notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_2b_kernels_exercise.ipynb
- Completed Notebook https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_2b_kernels_exercise.ipynb
- Day 0 Notebook (Container Testing): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_0_container_testing.ipynb
- Day 1A Notebook (Intro SPECFEM): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1a_intro_specfem2d.ipynb
- Day 1B Notebook (Fwd. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1b_forward_simulations.ipynb
- Day 2A Notebook (Adj. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_2a_kernels.ipynb

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.

-----------
## 0) Setting Up a SPECFEM2D Working Directory

- Let's set up a clean working directory to run SPECFEM2D  
- We will be doing all our work in the directory `/home/scoped/work/day_2/exercise`, all the following cells assume that we are in this directory  

In [ ]:
# Python packages we might use in this notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy.integrate import simps

In [ ]:
# Make correct dir. and move there
! mkdir -p /home/scoped/work/day_2/exercise
%cd /home/scoped/work/day_2/exercise

# Symlink the executables and copy the relevant DATA/ directory
! ln -s /home/scoped/specfem2d/bin .
! cp -r /home/scoped/specfem2d/EXAMPLES/Tape2007/DATA .
! cp -f DATA/Par_file_Tape2007_onerec DATA/Par_file

# Ensure that SPECFEM outputs required files for adjoint simulations
! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1

! mkdir OUTPUT_FILES

! ls

------------
## 1) Target Model Forward Simulations

- We'll use the homogeneous halfspace defined by default in the example as our target model
- We need to set up the working directory before running our simulation

### a) STATIONS
- Use the first 25 stations listed in the stations file 
- Or, using what you learned in Day 1, generate your own STATIONS file with an interesting configuration

### b) SOURCE
- Set one of the 25 sources listed in the DATA/ directory as your source file
- Remember that SPECFEM2D is expecting a file called SOURCE

### c) Run the Mesher and Solver  

1) Tell SPECFEM to use your STATIONS file, and not it's internal definition of stations
2) Run your simulation in **parallel** using 4 cores
    - *Remember* that you need to tell both SPECFEM and MPI that you are planning to run 4 processes  

The remainder of the `Par_file` should already be set up appropriately

### d) Save Results
- Make sure you **save the seismograms** output by SPECFEM somewhere safe  
- Subsequent simulations will **overwrite** files in the DATA/ and OUTPUT_FILES/ directory  


## 2) Initial Model Forward Simulations

- Now we want to edit the model definition to create a 'starting' model
- We'll use what we learned in the [Day 1B exercise](https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1b_forward_simulations.ipynb) to change the model parameters

### a) Edit Velocity Model
- The velocity model is defined in the `Par_file`
- **Decrease** the velocity values (Vp and Vs) of the starting model by 20\%  

### b) Edit the `Par_file`

1) Tell SPECFEM to save the forward wavefield after the simulation  
2) Tell SPECFEM to output binary database files 

### c) Run the Mesher, Solver and Save Results  

1) Run your simulation in **parallel** using 4 cores
2) After your simulation, **save the seismograms** output by SPECFEM somewhere safe  

## 3) Quantify Misfit, Generate Adjoint Sources

- You should now have **two sets of synthetics**, one generated by your initial model, another by your target model
- We now want to generate adjoint sources for each pair of synthetics

### a) Define your Misfit Function

- Let's use a cross correlation traveltime misfit function
- The cross correlation misfit is defined: $\chi (\mathbf{m}) = \frac{1}{2} \left[ T^{obs} - T(\mathbf{m}) \right] ^ 2$, 
- Where $T^{obs}$ is the observed traveltime, and $T(\mathbf{m})$ is the
predicted traveltime in Earth model $m$


>__Adjoint Source Equation:__ $f^{\dagger}(t) = - \left[ T^{obs} - T(\mathbf{m}) \right] ~ \frac{1}{N} ~
    \partial_t \mathbf{s}(T - t, \mathbf{m})$

### b) Generate Adjoint Sources

**Using a Python approach, for each pair of seismograms you must:**
1) Load in both data and synthetic seismogram (use `numpy.loadtxt`; see Day 2A; Section 2C)
2) Calculate the time shift $\left[ T^{obs} - T(\mathbf{m})\right]$ using the ObsPy's cross correlation function (https://docs.obspy.org/master/packages/autogen/obspy.signal.cross_correlation.correlate.html) 
    - The correlate function returns an array of correlation values
    - Use the `xcorr_max` to find the time shift related to the peak cross correlation
    - The time shift should be a **single value**
3) Assume the normalization factor $N$ is -1 (incorrect but simplified for convenience)
4) Differentiate the synthetic waveform, $\partial_t \mathbf{s}(t, \mathbf{m})$,  using NumPy diff (https://numpy.org/doc/stable/reference/generated/numpy.diff.html)
5) Time reverse and flip the differentiated synthetic waveform $\partial_t \mathbf{s}(T - t, \mathbf{m})$
6) Save the corresponding adjoint source in the `SEM/` directory (using `numpy.savetxt`; see Day 2A; Section 3A)
    - Remember to format the adjoint source the same as the input synthetics
    - Remember that adjoing sources must mimic the synthetic filename, but end with *.adj*

## 4) Run Adjoint Simulations

- Remember to tell SPECFEM that this is an adjoint simulation (not a forward simulation)
- Make sure that your adjoint sources are stored in the `SEM/` directory  
- Make sure your DATABASE files are available in the *OUTPUT_FILES/* directory
- Remember you do **not** need to run the mesher, only the solver
- Check the output log file and kernel files to make sure you ran an adjoint simulation (not forward)

## 5) Smooth Kernel
- Smooth your Vp and Vs kernels by 10km in the horizontal direction and 10 km in the vertical
- Make sure that SPECFEM can find the appropriate files
- Look at Day 2A Section 4 if you need help  

## 6) Visualize Results
- Use SeisFlows (or NumPy + Matplotlib) to visualize your kernel results
- Does your kernel make sense?
- Can you plot the sources and stations on top of the kernel figure?